In [9]:
import cv2
import numpy as np
import math

In [10]:
logo = cv2.imread('models/pyar_logo.png')

In [11]:
model = cv2.imread("reference/image.jpg", 1)
model = cv2.imread("reference/santo_patrono.jpg", 1)
stream = cv2.imread("reference/ref_smol.jpg", 1)


In [12]:
"""
SIFT - SURF - ORB
"""
model = cv2.imread("reference/image.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

frame = cv2.drawKeypoints(model, kp_model, None)
cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Keypoint in frame
"""

stream = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

frame = cv2.drawKeypoints(stream, kp_frame, None)
cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Try to find the ref
Aruco Marker
"""
model = cv2.imread("reference/image.jpg", 1)
frame = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# match frame descriptors with model descriptors
matches = bf.match(des_model, des_frame)

# sort them in the order of their distance
# the lower the distance, the better the match
matches = sorted(matches, key=lambda x: x.distance)

frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )


cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Santo patrono
"""
model = cv2.imread("reference/santo_patrono.jpg", 1)
frame = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# match frame descriptors with model descriptors
matches = bf.match(des_model, des_frame)
# sort them in the order of their distance
# the lower the distance, the better the match
matches = sorted(matches, key=lambda x: x.distance)

frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )


cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

Identificar key feachures con ORB BFMatcher y RANSAC<br>
Matriz de homografia para rotrotransladar el objeto

In [13]:
def projection_matrix(camera_parameters, homography):
    """
    From the camera calibration matrix and the estimated homography
    compute the 3D projection matrix
    """
    # Compute rotation along the x and y axis as well as the translation
    homography = homography * (-1)
    rot_and_transl = np.dot(np.linalg.inv(camera_parameters), homography)
    col_1 = rot_and_transl[:, 0]
    col_2 = rot_and_transl[:, 1]
    col_3 = rot_and_transl[:, 2]
    # normalise vectors
    l = math.sqrt(np.linalg.norm(col_1, 2) * np.linalg.norm(col_2, 2))
    rot_1 = col_1 / l
    rot_2 = col_2 / l
    translation = col_3 / l
    # compute the orthonormal basis
    c = rot_1 + rot_2
    p = np.cross(rot_1, rot_2)
    d = np.cross(c, p)
    rot_1 = np.dot(
        c / np.linalg.norm(c, 2) + d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_2 = np.dot(
        c / np.linalg.norm(c, 2) - d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_3 = np.cross(rot_1, rot_2)
    # finally, compute the 3D projection matrix from the model to the current frame
    projection = np.stack((rot_1, rot_2, rot_3, translation)).T
    return np.dot(camera_parameters, projection)


In [14]:
import cv2
import numpy as np
import math

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:

            # Drawing plane
            h, w = model.shape
            pts = np.float32(
                [
                 [-w*0.25, h * 0.25], 
                 [-w*0.25, h * 0.75], 
                 [w * 1.25, h * 0.75], 
                 [w* 1.25, h * 0.25]
                ]

            ).reshape(-1, 1, 2)
            # project corners into frame
            dst = cv2.perspectiveTransform(pts, homography)

            
            color = np.array([0, 204, 255])
            color = list(color[::-1])
            color = [int(c) for c in color]

            imgpts = np.int32(dst[:,:2])
            frame = cv2.fillConvexPoly(frame, imgpts, color)
            
            # connect them with lines
            frame = cv2.polylines(frame, [imgpts], True, 255, 3, cv2.LINE_AA)

            # obtain 3D projection matrix from homography matrix and camera parameters
            projection = projection_matrix(camera_parameters, homography)
            # project cube or model
            # frame = render(frame, obj, projection, model, False)
        """
        except:
            pass
        """
        # draw first 100 matches.
        
        """
        frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )
        """
        
        # show result
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



Not enough matches found - 58/75
Not enough matches found - 58/75


In [15]:
def projection_matrix(camera_parameters, homography):
    """
    From the camera calibration matrix and the estimated homography
    compute the 3D projection matrix
    """
    # Compute rotation along the x and y axis as well as the translation
    homography = homography * (-1)
    rot_and_transl = np.dot(np.linalg.inv(camera_parameters), homography)
    col_1 = rot_and_transl[:, 0]
    col_2 = rot_and_transl[:, 1]
    col_3 = rot_and_transl[:, 2]
    # normalise vectors
    l = math.sqrt(np.linalg.norm(col_1, 2) * np.linalg.norm(col_2, 2))
    rot_1 = col_1 / l
    rot_2 = col_2 / l
    translation = col_3 / l
    # compute the orthonormal basis
    c = rot_1 + rot_2
    p = np.cross(rot_1, rot_2)
    d = np.cross(c, p)
    rot_1 = np.dot(
        c / np.linalg.norm(c, 2) + d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_2 = np.dot(
        c / np.linalg.norm(c, 2) - d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_3 = np.cross(rot_1, rot_2)
    # finally, compute the 3D projection matrix from the model to the current frame
    projection = np.stack((rot_1, rot_2, rot_3, translation)).T
    return np.dot(camera_parameters, projection)

def render(img, obj, projection, model, color=False):
    """
    Render a loaded obj model into the current video frame
    """
    vertices = obj.vertices
    h, w = model.shape
     #Scale into view with scale matrix
    scale_matrix = np.eye(3) * 200

    for face in obj.faces:
        face_vertices = face[0]
        points = np.array([vertices[vertex - 1] for vertex in face_vertices])
        points = np.dot(points, scale_matrix)
        # render model in the middle of the reference surface. To do so,
        # model points must be displaced
        points = np.array([[p[0] + w / 2, p[1] + h / 2, p[2]] for p in points])
        dst = cv2.perspectiveTransform(points.reshape(-1, 1, 3), projection)
        imgpts = np.int32(dst)
        if color is False:
            cv2.fillConvexPoly(img, imgpts, (137, 27, 211))
        else:
            color = hex_to_rgb(face[-1])
            color = color[::-1]  # reverse
            cv2.fillConvexPoly(img, imgpts, color)

    return img

In [16]:
"""
Adding model
"""

import cv2
import numpy as np
import math

from objloader_simple import OBJ
fox = OBJ("models/fox.obj")

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:
            # obtain 3D projection matrix from homography matrix and camera parameters
            projection = projection_matrix(camera_parameters, homography)
            # project cube or model
            frame = render(frame, fox, projection, model, False)
        
        # show result
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



Not enough matches found - 58/75
Not enough matches found - 48/75


In [6]:
        
cap.release()
cv2.destroyAllWindows()